In [1]:
import csv
import math
from collections import OrderedDict

def calculate_euclidean_distance(x1, y1, x2, y2):
    """Calculate the Euclidean distance between two points."""
    return math.sqrt((x1 - x2)**2 + (y1 - y2)**2)

def load_grid(filename):
    """Load the grid from a CSV file."""
    grid = []
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            grid.append(row)
    return grid

def find_positions_of_char(grid, char):
    """Find all positions of a given character in the grid."""
    positions = []
    for i, row in enumerate(grid):
        for j, cell in enumerate(row):
            if cell == char:
                positions.append((i, j))
    return positions

def find_common_origins(grid, positions, distances):
    """Find common origin points that satisfy all distance constraints for the given character positions."""
    potential_origins = []
    size_x, size_y = len(grid), len(grid[0])
    
    # Check every point in the grid if it can be a common origin
    for x in range(size_x):
        for y in range(size_y):
            if all(math.isclose(calculate_euclidean_distance(x, y, pos[0], pos[1]), dist, abs_tol=0.1)
                   for pos, dist in zip(positions, distances)):
                potential_origins.append((x, y, grid[x][y]))  # Include the character at the origin
    
    return potential_origins

def validate_positions(grid, start_pos, target_char, required_dist):
    valid_positions = []
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == target_char and math.isclose(calculate_euclidean_distance(start_pos[0], start_pos[1], i, j), required_dist, abs_tol=0.1):
                valid_positions.append((i, j))
    return valid_positions

def find_valid_triplets(grid, distances):
    positions = {x: find_positions_of_char(grid, x) for x in distances.keys()}
    valid_triplets = []
    v1, v2, v3 = list(distances.keys())[:3]
    for start in positions[v1]:
        valid_p = validate_positions(grid, start, v2, distances[v1+v2])
        for p_pos in valid_p:
            valid_4 = validate_positions(grid, p_pos, v3, distances[v2+v3])
            for four_pos in valid_4:
                if math.isclose(calculate_euclidean_distance(start[0], start[1], four_pos[0], four_pos[1]), distances[v1+v3], abs_tol=0.1):
                    valid_triplets.append((start, p_pos, four_pos))
    return valid_triplets

def read_distances_in_chunks(filename):
    """Read the distances from a CSV file in chunks of six lines each."""
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile)
        chunk = []
        all_chunks = []
        for i, row in enumerate(reader):
            if len(row) == 2:  # Ensure the row is correctly formatted
                key, value = row[0], float(row[1])
                chunk.append((key, value))
                if (i + 1) % 6 == 0:  # Process in chunks of six
                    all_chunks.append(OrderedDict(chunk))
                    chunk = []
        if chunk:  # Add the last chunk if it doesn't complete six lines
            all_chunks.append(OrderedDict(chunk))
    return all_chunks

def main():
    grid = load_grid('grid.csv')
    chunks = read_distances_in_chunks('out.csv')
    for distances in chunks:
#     distances = OrderedDict([('[', 6.403124237), ('p', 5), ('4', 2.236067977), ('[p', 8), ('p4', 2.828427125), ('[4', 6.32455532)])
    # Get character positions from grid
        offset_chars = list(distances.keys())[:3]
#         print(distances)
#         print(offset_chars)
        char_positions = {char: find_positions_of_char(grid, char) for char in [offset_chars[1],offset_chars[2],offset_chars[0]]}
        # Define the distances as per the triplet mapping
        triplet_distances = [distances[offset_chars[0]], distances[offset_chars[1]], distances[offset_chars[2]]]

        # Find valid triplets first
        valid_triplets = find_valid_triplets(grid, distances)
        for triplet in valid_triplets:
            print("Valid triplet positions:", triplet)
            # Attempt to find common origins for each valid triplet
            origins = find_common_origins(grid, triplet, triplet_distances)
            print(f"Common origins for triplet {triplet}: {origins}")

if __name__ == "__main__":
    main()

Valid triplet positions: ((23, 66), (31, 66), (29, 68))
Common origins for triplet ((23, 66), (31, 66), (29, 68)): [(28, 70, 'H')]
Valid triplet positions: ((22, 82), (14, 82), (13, 71))
Common origins for triplet ((22, 82), (14, 82), (13, 71)): [(18, 76, 'T')]
Valid triplet positions: ((9, 31), (3, 32), (8, 33))
Common origins for triplet ((9, 31), (3, 32), (8, 33)): [(10, 29, 'B')]
Valid triplet positions: ((35, 8), (40, 11), (32, 6))
Common origins for triplet ((35, 8), (40, 11), (32, 6)): [(33, 4, '{')]
Valid triplet positions: ((11, 81), (0, 89), (11, 87))
Common origins for triplet ((11, 81), (0, 89), (11, 87)): [(4, 82, 's')]
Valid triplet positions: ((51, 91), (44, 80), (44, 79))
Common origins for triplet ((51, 91), (44, 80), (44, 79)): [(47, 85, 'Q')]
Valid triplet positions: ((80, 84), (76, 87), (71, 74))
Common origins for triplet ((80, 84), (76, 87), (71, 74)): [(74, 81, 'U')]
Valid triplet positions: ((91, 63), (87, 57), (81, 54))
Common origins for triplet ((91, 63), (87